# Fetch data

In [2]:
import snowflake.connector
import pandas as pd

In [4]:
start_date = '2024-06-14 11:00:00'
end_date = ' 2024-06-28 10:59:00'
campaign_name = 'global37_ADVERTISEMENT_Offerwall_ABN_tapjoy_GP_14_06_24'

In [5]:
sql_query = f'''WITH ab_test_activity_kpi AS (with crm as (
        select distinct user_id,
        trim(replace(value,'"',''),' ') AS campaign,
        left(campaign, position('.' in campaign) - 1) AS campaign_name,
        right(replace(value,'"',''), charindex('.', reverse(replace(value,'"',''))) - 1) AS campaign_group
        from
        (
        select *
        from GANGSTAR4.CRM_CONFIG_DOWNLOADED,
        lateral flatten(input=>split(campaign_list,','))
        where
         client_time >= '{start_date}'
         and client_time < '{end_date}'
         and left(trim(replace(value,'"',''),' '), position('.' in trim(replace(value,'"',''),' ')) - 1) = '{campaign_name}'
        )
      )
      , ua as (select *
      , min(active_date) over (partition by user_id,platform) as first_connection
      , max(active_date) over (partition by user_id,platform) as last_connection
      from "ELEPHANT_DB"."GANGSTAR4"."USER_ACTIVITY"
      where user_id in (select user_id from crm)
       and active_date >= to_date('{start_date}')
       and active_date < to_date('{end_date}')   )

      select crm.user_id
      , ua.platform
      , crm.campaign_name
      , campaign_group
      , active_date
      , install_date
      , revenue_today
      , revenue_ads_today
      , revenue_iap
      , IMPRESSIONS_ADS_TODAY
      , transactions_today
      , retention_day
      , is_pau_today
      , revenue_to_date
      , FIRST_IAP_DATE
      , FIRST_CONNECTION
      , LAST_CONNECTION
      , COUNTRY
      , install_source
      from crm
      inner join ua
      ON UA.user_id=CRM.user_id
      )
      select campaign_group, USER_ID, 
      sum(revenue_ads_today) as ADS_ARPU,
      sum(revenue_iap) as IAP_ARPU,
      (sum(revenue_ads_today)+sum(revenue_iap)) as ARPU,
      sum(IMPRESSIONS_ADS_TODAY) as AVG_impressions,
      CASE WHEN IAP_ARPU >0 THEN 1 ELSE 0 end as PAU,
      sum(revenue_ads_today)/COUNT(DISTINCT ACTIVE_DATE) as ADS_ARPDAU,
      sum(revenue_iap)/COUNT(DISTINCT ACTIVE_DATE) as IAP_ARPDAU,
      (ADS_ARPDAU+IAP_ARPDAU) as ARPDAU,
      sum(case when retention_day=1 then 1 else 0 end) as RETENTION_D1,
      sum(case when retention_day=3 then 1 else 0 end) as RETENTION_D3,
      sum(case when retention_day=7 then 1 else 0 end) as RETENTION_D7
      from ab_test_activity_kpi
      WHERE (ab_test_activity_kpi."CAMPAIGN_NAME" ) = '{campaign_name}'
      group by 1,2
      order by 1
'''

In [6]:
cursor = con.cursor()

cursor.execute(sql_query)
rows = 0
df_arr = []
while True:
    dat = cursor.fetchmany(50000)
    if not dat:
        break
    column_info = cursor.description
    column_names = [info[0] for info in column_info]
    
    df = pd.DataFrame(dat, columns=column_names)
    df_arr.append(df)
    rows += df.shape[0]
    
df = pd.concat(df_arr, ignore_index=True, sort=False)
df.to_csv('data.csv', index=False)

In [7]:
df.head()

,CAMPAIGN_GROUP,USER_ID,ADS_ARPU,IAP_ARPU,ARPU,AVG_IMPRESSIONS,PAU,ADS_ARPDAU,IAP_ARPDAU,ARPDAU,RETENTION_D1,RETENTION_D3,RETENTION_D7
0,Adjoy,5010049378876552964,0.009252,0.0,0.009252,4.0,0,0.009252,0.0,0.009252,0,0,0
1,Adjoy,5055365308984418102,0.007304,0.0,0.007304,8.0,0,0.001826,0.0,0.001826,0,0,0
2,Adjoy,8219131788899215889,0.010367,0.0,0.010367,5.0,0,0.010367,0.0,0.010367,0,0,0
3,Adjoy,553113818380189979,0.029190,0.0,0.029190,15.0,0,0.014595,0.0,0.014595,1,0,0
4,Adjoy,3474755085775092666,0.000622,0.0,0.000622,1.0,0,0.000622,0.0,0.000622,0,0,0


## Retention

In [8]:
first = f'''WITH ab_test_activity_kpi AS (with crm as (
        select distinct user_id,
        trim(replace(value,'"',''),' ') AS campaign,
        left(campaign, position('.' in campaign) - 1) AS campaign_name,
        right(replace(value,'"',''), charindex('.', reverse(replace(value,'"',''))) - 1) AS campaign_group
        from
        (
        select *
        from GANGSTAR4.CRM_CONFIG_DOWNLOADED,
        lateral flatten(input=>split(campaign_list,','))
        where
         client_time >= '{start_date}'
         and client_time < '{end_date}'
         and left(trim(replace(value,'"',''),' '), position('.' in trim(replace(value,'"',''),' ')) - 1) = '{campaign_name}'
        )
      )
      , ua as (select *
      , min(active_date) over (partition by user_id,platform) as first_connection
      , max(active_date) over (partition by user_id,platform) as last_connection
      from "ELEPHANT_DB"."GANGSTAR4"."USER_ACTIVITY"
      where user_id in (select user_id from crm)
       and active_date >= to_date('{start_date}')
       and active_date < to_date('{end_date}')   )

      select crm.user_id
      , ua.platform
      , crm.campaign_name
      , campaign_group
      , active_date
      , install_date
      , revenue_today
      , revenue_ads_today
      , revenue_iap
      , impressions_ads_today
      , transactions_today
      , retention_day
      , is_pau_today
      , revenue_to_date
      , FIRST_IAP_DATE
      , FIRST_CONNECTION
      , LAST_CONNECTION
      , COUNTRY
      , install_source
      from crm
      inner join ua
      ON UA.user_id=CRM.user_id
      )
'''


days = [1, 2,3,7,14, 30]
install_date_retention = [0]
list_str1 = []
for num in days:
    string = f"DIV0((COUNT(DISTINCT CASE WHEN retention_day  = {num} AND  (install_date between to_date('{start_date}') and to_date('{end_date}')) THEN user_id  ELSE NULL END)), (count(distinct case when install_date between to_date('{start_date}') and to_date('{end_date}') then user_id else null end )))  AS RETENTION_DAY_{num}"
    list_str1.append(string)

list_str2 = []
for num in days:
    string = f"COUNT(DISTINCT CASE WHEN retention_day  = {num} AND (install_date between to_date('{start_date}') and to_date('{end_date}')) THEN user_id  ELSE NULL END)  AS R{num}"
    list_str2.append(string)

list_str3 = []
for num in install_date_retention:
    string = f"COUNT(DISTINCT CASE WHEN retention_day  = {num} AND (install_date between to_date('{start_date}') and to_date('{end_date}')) THEN user_id  ELSE NULL END)  AS D0"
    list_str3.append(string)

second = 'SELECT campaign_group,\n' + ',\n'.join(list_str1) + ',\n'  + ',\n'.join(list_str2) + ',\n' + ',\n'.join(list_str3)

third = f'''
from ab_test_activity_kpi
WHERE ("CAMPAIGN_NAME" ) = '{campaign_name}'
GROUP BY 1'''


In [9]:
cursor = con.cursor()

retention_query = first+second+third

cursor.execute(retention_query)
rows = 0
df_arr = []
while True:
    dat = cursor.fetchmany(50000)
    if not dat:
        break
    column_info = cursor.description
    column_names = [info[0] for info in column_info]
    
    df = pd.DataFrame(dat, columns=column_names)
    df_arr.append(df)
    rows += df.shape[0]
    
df_retention = pd.concat(df_arr, ignore_index=True, sort=False)
df_retention.to_csv('data_retention.csv', index=False)

In [10]:
df_retention

,CAMPAIGN_GROUP,RETENTION_DAY_1,RETENTION_DAY_2,RETENTION_DAY_3,RETENTION_DAY_7,RETENTION_DAY_14,RETENTION_DAY_30,R1,R2,R3,R7,R14,R30,D0
0,Tapjoy,0.634804,0.462929,0.362853,0.160151,0.000000,0.000000,73968,53941,42280,18661,0,0,116521
1,Adjoy,0.635741,0.463913,0.365088,0.157881,0.000000,0.000000,73568,53684,42248,18270,0,0,115720


# Processing data

In [11]:
from scipy import stats
from statsmodels.stats.proportion import proportions_ztest
import numpy as np

In [12]:
df_indiv = pd.read_csv('data.csv')
df_indiv.fillna(0, inplace=True)
df_retention = pd.read_csv('data_retention.csv')
df_retention.fillna(0, inplace=True)

In [13]:
columns = ['ADS_ARPDAU', 'IAP_ARPDAU', 'ARPDAU',  'IAP_ARPU', 'ADS_ARPU','ARPU', 'AVG_IMPRESSIONS', 'RETENTION_DAY_1', 'RETENTION_DAY_7', 'RETENTION_DAY_14', 'RETENTION_DAY_30']
groups = ['Adjoy', 'Tapjoy']


pval = []

for col in columns:
    if 'RETENTION' in col:
        number = col.split('_')[-1]
        R = 'R' + number
        D = 'D0'
        
        count = np.array([df_retention[(df_retention['CAMPAIGN_GROUP'] == groups[0]) ][R], 
                         df_retention[(df_retention['CAMPAIGN_GROUP'] == groups[1]) ][R]])
        nobs = np.array([df_retention[(df_retention['CAMPAIGN_GROUP'] == groups[0]) ][D], 
                         df_retention[(df_retention['CAMPAIGN_GROUP'] == groups[1]) ][D]])
        stat, pv = proportions_ztest(count, nobs)

        control_mean = df_retention[df_retention['CAMPAIGN_GROUP'] == groups[0]][col].mean(skipna=False)
        ws1_mean = df_retention[df_retention['CAMPAIGN_GROUP'] == groups[1]][col].mean(skipna=False)

        pval.append([col, control_mean, ws1_mean, pv[0]])

    else:
        control_mean = df_indiv[df_indiv['CAMPAIGN_GROUP'] == groups[0]][col].mean(skipna=False)
        ws1_mean = df_indiv[df_indiv['CAMPAIGN_GROUP'] == groups[1]][col].mean(skipna=False)

        stat, pv = stats.mannwhitneyu(
            df_indiv[(df_indiv['CAMPAIGN_GROUP'] == groups[0])][col],
            df_indiv[(df_indiv['CAMPAIGN_GROUP'] == groups[1])][col],
            alternative= 'two-sided'
        )

        pval.append([col, control_mean, ws1_mean, pv])




C:\Users\anh.dongoc\AppData\Roaming\Python\Python311\site-packages\statsmodels\stats\weightstats.py:792: RuntimeWarning: invalid value encountered in divide
  zstat = value / std


In [14]:
df_pval_adjoe_tapjoy = pd.DataFrame(pval,columns=['Metric user_id',groups[0], groups[1], 'p_value_adjoe_tapjoy'])
df_pval_adjoe_tapjoy

,Metric user_id,Adjoy,Tapjoy,p_value_adjoe_tapjoy
0,ADS_ARPDAU,0.009605,0.009556,0.044407
1,IAP_ARPDAU,0.003448,0.003045,0.484960
2,ARPDAU,0.011921,0.011683,0.041374
3,IAP_ARPU,0.015584,0.015353,0.484404
4,ADS_ARPU,0.045824,0.045211,0.070833
5,ARPU,0.056304,0.056391,0.066012
6,AVG_IMPRESSIONS,21.419119,21.172194,0.004226
7,RETENTION_DAY_1,0.635741,0.634804,0.638890
8,RETENTION_DAY_7,0.157881,0.160151,0.134681
9,RETENTION_DAY_14,0.000000,0.000000,NaN


In [16]:
writer = pd.ExcelWriter('Adjoy_GGplay_ABN.xlsx') 
my_sheet_name = campaign_name[:21]
df_excel = df_pval_adjoe_tapjoy.round(3)

df_excel.to_excel(writer, sheet_name= my_sheet_name, index=False) 

writer.sheets[my_sheet_name].set_column(0, 0, 22)
writer.close()